In [2]:
from front.util import deep_merge

d1 = {'a':{'b':{'c':12},'d':32}}
d1

{'a': {'b': {'c': 12}, 'd': 32}}

In [7]:
d2 = {'a':{'b':'new value'}}
d3 = {'a':{'b':{'d':'new value'}}}
d4 = {'e':{'f':54}}


In [8]:
deep_merge(d1,d2)
deep_merge(d1,d3)
deep_merge(d1,d4)

{'a': {'b': {'c': 12}, 'd': 32}, 'e': {'f': 54}}

In [9]:
data = {'a': {'b': {'c': 'value'}}}
value = getattr(data, 'a.b.c')
print(value) # Output: 'value'


AttributeError: 'dict' object has no attribute 'a.b.c'

In [15]:
''.join(['a', 'b', 'c'])


'abc'

In [17]:
from know.boxes import *
from front import APP_KEY, RENDERING_KEY, ELEMENT_KEY
from streamlitfront import mk_app, binder as b
from typing import Mapping
from i2 import Sig
from front.dag import crudify_funcs
from meshed import code_to_dag

from streamlitfront.elements import SelectBox


def foo(x, y, z):
    return int(x) + int(y) * int(z)


@code_to_dag
def dag():
    res = first_func(x, y, z)
    result = second_func(res)


dag = dag.ch_funcs(first_func=foo, second_func=lambda res: result)

/Users/sylvain/Desktop/dev/otosense/slang/slang/util.py:95: RuntimeWarning: invalid value encountered in cast
  snip_of_unichr_code = (nan * ones(unichr_code_of_snip.max() + 1)).astype(int)
<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
/Users/sylvain/Desktop/dev/otosense/streamlitfront/streamlitfront/session_state.py:10: UserWarning: Use streamlit 1.11.1 if you want to use the old and deprecated dispatch_funcs function.
  warn('Use streamlit 1.11.1 if you want to use the old and deprecated dispatch_funcs function.')


In [18]:
vars(dag)

{'func_nodes': [FuncNode(x,y,z -> first_func -> res),
  FuncNode(res -> second_func -> result)],
 'cache_last_scope': True,
 'parameter_merge': <function meshed.util.conservative_parameter_merge(params, same_kind=True, same_default=True, same_annotation=True)>,
 'new_scope': dict,
 'name': None,
 'extract_output_from_scope': <function meshed.util.extract_values(d: dict, keys: Iterable)>,
 'graph': {'x': [FuncNode(x,y,z -> first_func -> res)],
  'y': [FuncNode(x,y,z -> first_func -> res)],
  'z': [FuncNode(x,y,z -> first_func -> res)],
  FuncNode(x,y,z -> first_func -> res): ['res'],
  'res': [FuncNode(res -> second_func -> result)],
  FuncNode(res -> second_func -> result): ['result']},
 'nodes': ['x',
  'y',
  'z',
  FuncNode(x,y,z -> first_func -> res),
  'res',
  FuncNode(res -> second_func -> result),
  'result'],
 'var_nodes': ['x', 'y', 'z', 'res', 'result'],
 '__signature__': <Sig (x, y, z)>,
 'roots': ('x', 'y', 'z'),
 'leafs': ('result',),
 'last_scope': None,
 '__name__': 'DA

In [20]:
from front.dag import crudify_funcs
mall = dict()
_funcs = crudify_funcs(var_nodes="res", dag=dag, mall=mall)


In [22]:
len(_funcs)

2

In [23]:
_funcs[1]

<i2.Wrap <lambda>(res: Literal[])>

In [24]:
from i2 import Wrap

In [25]:
Wrap??

Init signature: Wrap(func, ingress=None, egress=None, *, name=None)
Source:        
class Wrap(_Wrap):
    """A function wrapper with interface modifiers.

    :param func: The wrapped function
    :param ingress: The incoming data transformer. It determines the argument properties
        (name, kind, default and annotation) as well as the actual input of the
        wrapped function.
    :param egress: The outgoing data transformer. It also takes precedence over the
        wrapped function to determine the return annotation of the ``Wrap`` instance
    :param name: Name to give the wrapper (will use wrapped func name by default)
    :return: A callable instance wrapping ``func``

    Some examples:

    >>> from inspect import signature
    >>> from i2 import Sig

    >>> def func(a, b):
    ...     return a * b

    >>> wrapped_func = wrap(func)  # no transformations: wrapped_func is the same as func
    >>> assert wrapped_func(2, 'Hi') == func(2, 'Hi') == 'HiHi'

    Modifying the